# Assignment 06
EE 538 Spring 2020 <br>
Analog Circuits for Sensor Systems <br>
University of Washington Electrical & Computer Engineering <br>

Due: May 23, 2020 <br>
Author: Kevin Egedy

In [1]:
# Imports
import os
import sys
import cmath
import math
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd
import ltspice
import sympy as sp
from scipy import signal
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
def read_ltspice(file_name,ftype='trans',units='db'):
    cols = []
    arrs = []
    with open(file_name, 'r',encoding='utf-8') as data:
        for i,line in enumerate(data):
            if i==0:
                cols = line.split()
                arrs = [[] for _ in cols]
                continue
            parts = line.split()
            for j,part in enumerate(parts):
                arrs[j].append(part)
    df = pd.DataFrame(arrs,dtype='float64')
    df = df.T
    df.columns = cols
    if ftype=='trans':
        return df
    elif ftype=='ac':
        if units=='db':
            for col in cols:
                if df[col].str.contains(',').all():
                    df[f'Mag_{col}'] = df[col].apply(lambda x: x.split(',')[0])
                    df[f'Mag_{col}'] = df[f'Mag_{col}'].apply(lambda x: x[1:-2])
                    df[f'Mag_{col}'] = df[f'Mag_{col}'].astype('float64')
                    df[f'Phase_{col}'] = df[col].apply(lambda x: x.split(',')[1])
                    df[f'Phase_{col}'] = df[f'Phase_{col}'].apply(lambda x: x[0:-2])
                    df[f'Phase_{col}'] = df[f'Phase_{col}'].astype('float64')
        if units=='cartesian':
            for col in cols:
                if df[col].str.contains(',').all():
                    df[f'Re_{col}'] = df[col].apply(lambda x: x.split(',')[0])
                    df[f'Re_{col}'] = df[f'Re_{col}'].astype('float64')
                    df[f'Im_{col}'] = df[col].apply(lambda x: x.split(',')[1])
                    df[f'Im_{col}'] = df[f'Im_{col}'].astype('float64')
        df['Freq.'] = df['Freq.'].astype('float64')
        return df
    else:
        print('invalid ftype')

In [6]:
# Area Approximation

def AreaApprox(f,a,b,n,offset):
    '''
    Returns the integral approximation of f(x)dx from a to b
    f : continuous waveform
    a : starting point
    b : ending point
    '''
    apprx = 0
    dxs,width = np.linspace(a,b,n,endpoint=False,retstep=True)
    for dx in dxs:
        dx = int(dx)
        midpoint = (f[dx]+f[dx+1])/2 # Use the midpoint approximation
        apprx += width*(midpoint-offset) # width is 1 second -> adjust by resolution
    return apprx[0]

<img src="images/HW06Q1.png" width="700">

**Input-Referred Voltage Noise**

<img src="images/input_voltage_noise.png" width="400">

<hr>

**Input Resistor Noise**

<img src="images/input_resistor_noise.png" width="300">

$$\begin{eqnarray}
&& e_{n1,out} &=& \frac{-R_2}{R_1}e_{n1} \\[0.5em]
&& e_{n1,out}^2 &=& (\frac{R_2}{R_1})^2 4kTR_1 && \frac{V^2}{\text{Hz}}
\end{eqnarray}$$

<hr>

**Feedback Resistor Noise**

<img src="images/feedback_resistor_noise.png" width="300">

$$\begin{eqnarray}
&& e_{n2,out} &=& e_{n2} \\[0.5em]
&& e_{n2,out}^2 &=& 4kTR_1 && \frac{V^2}{\text{Hz}}
\end{eqnarray}$$

<hr>

**Opamp Noise**

<img src="images/opamp_noise.png" width="300">

$$\begin{eqnarray}
&& e_{n,out} &=& (1+\frac{R_2}{R_1})e_{na} \\[0.5em]
&& e_{na,out}^2 &=& (1+\frac{R_2}{R_1})^2 e_{na}^2 && \frac{V^2}{\text{Hz}}
\end{eqnarray}$$

<hr>

**Part A**

**Given**: $e_n = \frac{10nV}{\sqrt{\text{Hz}}}, i_n = \frac{1pA}{\sqrt{\text{Hz}}}, f_T = 10\text{MHz}$

$\begin{eqnarray}
&& A_V &=& \frac{1}{\beta} = \frac{-R_f}{R_i} = 10 \\[0.5em]
&& v_{n,in} &\approx& \frac{v_{n,out}}{1/\beta} \\[0.5em]
\end{eqnarray}$

$\begin{eqnarray}
&& e_{n,out} &=& \frac{-R_f}{R_i}e_{n} + e_{n} + \\[0.5em]
&& e_{n,out} &=& e_{n}(\frac{-R_f}{R_i} + 1) \\[0.5em]
&& e_{n,out} &=& 10\cdot10^{-9} (-10 + 1) \\[0.5em]
&& e_{n,out} &=& -90 nV \\[0.5em]
\end{eqnarray}$

<hr>

**Part B**

$\begin{eqnarray}
&& A_V &=& \frac{1}{\beta} = \frac{-R_f}{R_i} = 10 \\[0.5em]
&& v_{n,out(rms)} &=& \sqrt{e_{n,out}^2 \frac{\pi}{2}\beta f_T} \\[0.5em]
&& v_{n,in(rms)} &\approx& \frac{v_{n,out(rms)}}{1/\beta} \\[0.5em]
&& &=& \beta \sqrt{e_{n,out}^2 \frac{\pi}{2}f_T} \\[0.5em]
&& &=& \frac{1}{10} \sqrt{\frac{\pi}{2} (-90\cdot10^{-9})^2 (10\cdot10^6}) =
       35.7 \frac{mV^2}{\text{Hz}} = 0.00000357 \frac{V^2}{\text{Hz}} \\[0.5em]
\end{eqnarray}$

<hr>

**Part C**

<img src="images/HW06Q2.png" width="700">

**Part A**

$$\begin{align}
  \begin{array}{ll}
 f_{3dB} \approx \beta f_T && f_{\text{ENB}} = \frac{\pi}{2}f_{3dB}
  \end{array}
\end{align}$$